In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn.preprocessing import normalize

In [164]:
xy = np.loadtxt('C:\\Users\\SANHA\\Desktop\\train100000.csv', delimiter=',', dtype=np.float32)
xy3 = np.loadtxt('C:\\Users\\SANHA\\Desktop\\1111.csv', delimiter=',', dtype=np.float32)
x_data=xy[:,0:-1]
x_data = normalize(x_data, axis=0, norm='max')
xy[:,0:-1]=x_data
y_data=xy[:,[-1]]
m_data=xy[:499,:]
r_data=xy[:10000,:]
g_data=xy[:0,:]
F_data=xy[:,:]
F2_data=xy3[:,:]

In [103]:
# Model Inputs
def model_inputs(real_dim, noise_dim):
    inputs_real_ = tf.placeholder(tf.float32, shape=[None, real_dim], name='inputs_real')
    inputs_z_ = tf.placeholder(tf.float32, shape=[None, noise_dim], name='inputs_z')
    
    return inputs_real_, inputs_z_

def leaky_relu(x, alpha):
    return tf.maximum(alpha * x, x)

In [104]:
# Generator Network
def model_generator(z_input, out_dim, n_units=128, reuse=False, alpha=0.01):
    # used to reuse variables, name scope
    with tf.variable_scope('generator', reuse=reuse):
        hidden_layer = tf.layers.dense(z_input, n_units, activation=None)
        hidden_layer = leaky_relu(hidden_layer, alpha)
        
        logits = tf.layers.dense(hidden_layer, out_dim, activation=None)
        outputs = tf.nn.sigmoid(logits)
        
        return outputs, logits

In [105]:
# Discriminator Network
def model_discriminator(input, n_units=128, reuse=False, alpha=0.1):
    with tf.variable_scope('discriminator', reuse=reuse):
        hidden_layer = tf.layers.dense(input, n_units, activation=tf.nn.relu)
        #hidden_layer = leaky_relu(hidden_layer, alpha)
        
        logits = tf.layers.dense(hidden_layer, 1, activation=None)
        outputs = tf.nn.sigmoid(logits)
        
        return outputs, logits

In [106]:
input_size = 42
z_dim = 21
g_hidden_size = 128
d_hidden_size = 128
alpha = 0.1
smooth = 0.1
learning_rate = 0.001

In [107]:
tf.reset_default_graph()  # If we don't have this, as we call this block over and over, the graph gets bigger and bigger

graph = tf.Graph()
with graph.as_default():
    inputs_real, inputs_z = model_inputs(input_size, z_dim)
    
    g_outputs, g_logits = model_generator(inputs_z, input_size, n_units=g_hidden_size, reuse=False, alpha=alpha)
    
    d_outputs_real, d_logits_real = model_discriminator(inputs_real, n_units=d_hidden_size, reuse=False, alpha=alpha)
    d_outputs_fake, d_logits_fake = model_discriminator(g_outputs, n_units=d_hidden_size, reuse=True, alpha=alpha)
    
    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real, labels=tf.ones_like(d_logits_real) * (1-smooth)))
    d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.zeros_like(d_logits_fake)))
    
    d_loss = d_loss_real + d_loss_fake
    g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.ones_like(d_logits_fake)))
    
    t_vars = tf.trainable_variables()
    g_vars = [variable for variable in t_vars if 'generator' in variable.name]
    d_vars = [variable for variable in t_vars if 'discriminator' in variable.name]
    
    # Affected Variables with var_list
    d_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(d_loss, var_list=d_vars)
    g_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(g_loss, var_list=g_vars)
    
    # Saving variables with var_list
    saver = tf.train.Saver(var_list=g_vars)

In [146]:
f = open('C:\\Users\\SANHA\\Desktop\\train10_n.csv', 'w')
for step in range(100000):
    f.write("%f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f \n" %(xy[step,0],xy[step,1],xy[step,2],xy[step,3],xy[step,4],xy[step,5],xy[step,6],xy[step,7],xy[step,8],xy[step,9],xy[step,10],xy[step,11],xy[step,12],xy[step,13],xy[step,14],xy[step,15],xy[step,16],xy[step,17],xy[step,18],xy[step,19],xy[step,20],xy[step,21],xy[step,22],xy[step,23],xy[step,24],xy[step,25],xy[step,26],xy[step,27],xy[step,28],xy[step,29],xy[step,30],xy[step,31],xy[step,32],xy[step,33],xy[step,34],xy[step,35],xy[step,36],xy[step,37],xy[step,38],xy[step,39],xy[step,40],xy[step,41]))
        
f.close()

In [167]:
xy2 = np.loadtxt('C:\\Users\\SANHA\\Desktop\\test20000.csv', delimiter=',', dtype=np.float32)
tx_data=xy2[:,0:-1]
tx_data = normalize(tx_data, axis=0, norm='max')
xy2[:,0:-1]=tx_data
ty_data=xy2[:,[-1]]

In [144]:
samples=[]
normal=0
abnormal=0
count=0
#line_sample=[1][42]
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    f = open('C:\\Users\\SANHA\\Desktop\\gen_sample.txt', 'w')
    for step in range(100000):
        if count==308:
            count=0
        batch_images = F2_data[count].reshape([1, 42])
        count+=1
        batch_z = np.random.uniform(-1, 1, size=[1, z_dim])
        
        _ = sess.run(d_optimizer, feed_dict={inputs_real : batch_images, inputs_z : batch_z})
        _ = sess.run(g_optimizer, feed_dict={inputs_z : batch_z})
        loss_d, loss_g = sess.run([d_loss, g_loss], feed_dict={inputs_real : batch_images, inputs_z : batch_z})
        #if step%1000==0:
        #    print('step {} / {} Complete. D_Loss : {:0.3f}, G_Loss : {:0.3f}'.format(step+1, 100000, loss_d, loss_g))
        sys.stdout.write("\r%d / %d: %f, %f" % (step, 100000, loss_d, loss_g))
        sys.stdout.flush()
        sample_z = np.random.uniform(-1, 1, size=[1, z_dim])  # 16 Samples each epoch
        gen_samples, _ = sess.run(model_generator(inputs_z, input_size, reuse=True), feed_dict={inputs_z : sample_z})
        
        temp=gen_samples[0,41]
        #print(temp)
        
        #print(temp,gen_samples[0,41])
        f.write("%f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f \n" %(gen_samples[0,0],gen_samples[0,1],gen_samples[0,2],gen_samples[0,3],gen_samples[0,4],gen_samples[0,5],gen_samples[0,6],gen_samples[0,7],gen_samples[0,8],gen_samples[0,9],gen_samples[0,10],gen_samples[0,11],gen_samples[0,12],gen_samples[0,13],gen_samples[0,14],gen_samples[0,15],gen_samples[0,16],gen_samples[0,17],gen_samples[0,18],gen_samples[0,19],gen_samples[0,20],gen_samples[0,21],gen_samples[0,22],gen_samples[0,23],gen_samples[0,24],gen_samples[0,25],gen_samples[0,26],gen_samples[0,27],gen_samples[0,28],gen_samples[0,29],gen_samples[0,30],gen_samples[0,31],gen_samples[0,32],gen_samples[0,33],gen_samples[0,34],gen_samples[0,35],gen_samples[0,36],gen_samples[0,37],gen_samples[0,38],gen_samples[0,39],gen_samples[0,40],gen_samples[0,41]))
       
    
        
    print('Generating Complete. normal={}, abnormal={}'.format(normal,abnormal))
    f.close()

20470 / 100000: 1.486603, 0.867135

KeyboardInterrupt: 

In [148]:
xy2 = np.loadtxt('C:\\Users\\SANHA\\Desktop\\train_11.csv', delimiter=',', dtype=np.float32)
#xy3 = np.loadtxt('C:\\Users\\SANHA\\Desktop\\test.csv', delimiter=',', dtype=np.float32)
#xy4 = np.loadtxt('C:\\Users\\SANHA\\Desktop\\test1.csv', delimiter=',', dtype=np.float32)



#false data

Fx_data=xy2[:,0:-1]
Fy_data=xy2[:,[-1]]
print("finished")

finished


In [174]:
#초기화
normal= 0
apache2=0
back =0
buffer_overflow =0
ftp_write =0
guess_passwd =0
httptunnel =0
imap =0
ipsweep =0
land =0
loadmodule =0
mailbomb =0
mscan =0
multihop =0
named =0
neptune =0
nmap =0
perl =0
phf =0
pod =0
portsweep =0
processtable =0
ps =0
rootkit =0
saint =0
satan =0
sendmail =0
smurf =0
snmpgetattack =0
snmpguess =0
sqlattack =0
teardrop =0
udpstorm =0
warezmaster =0
warezclient =0
worm =0
xlock =0
xsnoop =0
xterm =0
spy =0

#분류
nb_classes=40

X=tf.placeholder(tf.float32,[None,41])
Y=tf.placeholder(tf.int32,[None,1])

Y_one_hot=tf.one_hot(Y,nb_classes)
Y_one_hot=tf.reshape(Y_one_hot,[-1,nb_classes])

W1=tf.Variable(tf.random_normal([41,41]),name='weight1')
b1=tf.Variable(tf.random_normal([41]),name='bias1')
layer1=tf.sigmoid(tf.matmul(X,W1)+b1)

W2=tf.Variable(tf.random_normal([41,41]),name='weight2')
b2=tf.Variable(tf.random_normal([41]),name='bias2')
layer2=tf.sigmoid(tf.matmul(layer1,W2)+b2)

W3=tf.Variable(tf.random_normal([41,nb_classes]),name='weight3')
b3=tf.Variable(tf.random_normal([nb_classes]),name='bias3')
logits=tf.matmul(layer2,W3)+b3
hypothesis=tf.nn.softmax(logits)

cost_i=tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y_one_hot)

cost=tf.reduce_mean(cost_i)
optimizer=tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

prediction=tf.argmax(hypothesis,1) #가능성을 퍼센트로~~
correct_prediction=tf.equal(prediction,tf.arg_max(Y_one_hot,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    f = open('C:\\Users\\SANHA\\Desktop\\false_data.txt', 'a+')
    for step in range(1001):
        sess.run(optimizer,feed_dict={X:Fx_data,Y:Fy_data})
        #print("training by gan sample")
        if step %10==0:
            loss,acc=sess.run([cost,accuracy],feed_dict={X:Fx_data,Y:Fy_data})
            print("step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step,loss,acc))
  
    
#m_data=np.append(m_data,gen_samples,axis=0)

    tr=0
    fa=0
    total=0
    pred = sess.run(prediction, feed_dict={X: tx_data})
    for p, y in zip(pred, ty_data.flatten()):
            if(p==int(y)):
                tr=tr+1
                total=total+1
            else:
                fa=fa+1
                if(int(y)==0):
                    normal+=1
                    
                elif(int(y)==1):
                    apache2+=1
                elif(int(y)==2):
                    back+=1

                elif(int(y)==3):
                    buffer_overflow+=1
                elif(int(y)==4):
                    ftp_write+=1
                elif(int(y)==5):
                    guess_passwd+=1
                elif(int(y)==6):
                    httptunnel+=1
                elif(int(y)==7):
                    imap+=1
                elif(int(y)==8):
                    ipsweep+=1
                elif(int(y)==9):
                    land+=1
                elif(int(y)==10):
                    loadmodule+=1
                elif(int(y)==11):
                    mailbomb+=1
                elif(int(y)==12):
                    mscan+=1
                elif(int(y)==13):
                    multihop+=1
                elif(int(y)==14):
                    named+=1
                elif(int(y)==15):
                    neptune+=1
                elif(int(y)==16):
                    nmap+=1
                elif(int(y)==17):
                    perl+=1
                elif(int(y)==18):
                    phf+=1
                elif(int(y)==19):
                    pod+=1
                elif(int(y)==20):
                    portsweep+=1
                elif(int(y)==21):
                    processtable+=1
                elif(int(y)==22):
                    ps+=1
                elif(int(y)==23):
                    rootkit+=1
                elif(int(y)==24):
                    saint+=1
                elif(int(y)==25):
                    satan+=1
                elif(int(y)==26):
                    sendmail+=1
                elif(int(y)==27):
                    smurf+=1
                elif(int(y)==28):
                    snmpgetattack+=1
                elif(int(y)==29):
                    snmpguess+=1
                elif(int(y)==30):
                    sqlattack+=1
                elif(int(y)==31):
                    teardrop+=1
                elif(int(y)==32):
                    udpstorm+=1
                elif(int(y)==33):
                    warezmaster+=1
                elif(int(y)==34):
                    warezclient+=1
                    f.write("%f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f \n" \
                        %(tx_data[total,0],tx_data[total,1],tx_data[total,2],tx_data[total,3],tx_data[total,4],tx_data[total,5],tx_data[total,6],tx_data[total,7],tx_data[total,8],tx_data[total,9],tx_data[total,10],tx_data[total,11],tx_data[total,12],tx_data[total,13],tx_data[total,14],tx_data[total,15],tx_data[total,16],tx_data[total,17],tx_data[total,18],tx_data[total,19],tx_data[total,20],tx_data[total,21],tx_data[total,22],tx_data[total,23],tx_data[total,24],tx_data[total,25],tx_data[total,26],tx_data[total,27],tx_data[total,28],tx_data[total,29],tx_data[total,30],tx_data[total,31],tx_data[total,32],tx_data[total,33],tx_data[total,34],tx_data[total,35],tx_data[total,36],tx_data[total,37],tx_data[total,38],tx_data[total,39],tx_data[total,40],ty_data[total,0]))

                elif(int(y)==35):
                    worm+=1
                elif(int(y)==36):
                    xlock+=1
                elif(int(y)==37):
                    xsnoop+=1
                elif(int(y)==38):
                    xterm+=1
                elif(int(y)==39):
                    spy+=1

                total=total+1



                #print("[{}] Prediction: {} Real Y: {}".format(p == int(y), p, int(y)))
       
    print("true={} false: {} acc: {:0.4f}".format(tr,fa,tr/(tr+fa)))
    print("normal={},apache2={},back={},buffer_overflow={},ftp_write={},guess_passwd={},httptunnel={},imap={},ipsweep={},land={},loadmodule={},mailboml={},mscan={},multihop={},named={},neptune={},nmap={},perl={},phf={},pod={},portsweep={},processtable={},ps={},rootkit={},saint={},satan={},sendmail={},smurf={},snmpgetattack={},snmpguess={},sqlattack={},teardrop={},updstorm={},warezmaster={},warezclient={},worm={},xlock={},\
    xsnoop={},xterm={},spy={}".format(normal,apache2,back,buffer_overflow,ftp_write,guess_passwd,httptunnel,imap,ipsweep,land,loadmodule,mailbomb,mscan,multihop,named,neptune,nmap,perl,phf,pod,portsweep,processtable,ps,rootkit,saint,satan,sendmail,smurf,snmpgetattack,snmpguess,sqlattack,teardrop\
                  ,udpstorm,warezmaster,warezclient,worm,xlock,xsnoop,xterm,spy))
    f.close()

step:     0	Loss: 6.614	Acc: 0.00%
step:    10	Loss: 1.377	Acc: 74.94%
step:    20	Loss: 0.817	Acc: 84.77%
step:    30	Loss: 0.539	Acc: 89.57%
step:    40	Loss: 0.411	Acc: 89.66%
step:    50	Loss: 0.338	Acc: 89.98%
step:    60	Loss: 0.279	Acc: 93.21%
step:    70	Loss: 0.236	Acc: 94.00%
step:    80	Loss: 0.208	Acc: 94.65%
step:    90	Loss: 0.188	Acc: 95.07%
step:   100	Loss: 0.173	Acc: 95.24%
step:   110	Loss: 0.160	Acc: 95.44%
step:   120	Loss: 0.150	Acc: 95.62%
step:   130	Loss: 0.141	Acc: 95.79%
step:   140	Loss: 0.132	Acc: 95.97%
step:   150	Loss: 0.125	Acc: 96.14%
step:   160	Loss: 0.118	Acc: 96.29%
step:   170	Loss: 0.112	Acc: 96.41%
step:   180	Loss: 0.107	Acc: 96.49%
step:   190	Loss: 0.102	Acc: 96.57%
step:   200	Loss: 0.098	Acc: 96.65%
step:   210	Loss: 0.094	Acc: 96.84%
step:   220	Loss: 0.090	Acc: 97.01%
step:   230	Loss: 0.085	Acc: 97.21%
step:   240	Loss: 0.081	Acc: 97.47%
step:   250	Loss: 0.078	Acc: 97.59%
step:   260	Loss: 0.075	Acc: 97.72%
step:   270	Loss: 0.072	Acc: 